In [ ]:
from random import seed
from random import randrange
from csv import reader

random - returns a random float number b/w 0 and 1

seed - initialize the random number generator

csv - a type of plain text file that uses specific structurinf to 

arrange tabular data

randrange - return a random no b/w the given range

reader - it is used to read a file

The banknote dataset involves predicting whether a given banknote is authentic given a number of measures taken from a photograph

In [ ]:
# Load a CSV file
def load_csv(filename):
 file = open(filename, "rt")
 lines = reader(file)
 dataset = list(lines)
 return dataset

str_column_to_float() to convert string numbers to floats.

In [ ]:
# Convert string column to float
def str_column_to_float(dataset, column):
 for row in dataset:
  row[column] = float(row[column].strip())

We will evaluate the algorithm using k-fold cross-validation with 5 folds. This means that 1372/5=274.4 or just over 270 records will be used in each fold.

In [ ]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
 dataset_split = list()
 dataset_copy = list(dataset)
 fold_size = int(len(dataset) / n_folds)
 for i in range(n_folds):
  fold = list()
 while len(fold) < fold_size:
  index = randrange(len(dataset_copy))
 fold.append(dataset_copy.pop(index))
 dataset_split.append(fold)
 return dataset_split

accuracy_metric() to calculate the accuracy of predictions.

In [ ]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
 correct = 0
 for i in range(len(actual)):
  if actual[i] == predicted[i]:
    correct += 1
 return correct / float(len(actual)) * 100.0

helper functions evaluate_algorithm() to evaluate the algorithm with cross-validation 

In [ ]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds=5,max_depth=5,min_size=10, *args):
 folds = cross_validation_split(dataset, n_folds)
 scores = list()
 for fold in folds:
  train_set = list(folds)
 train_set.remove(fold)
 train_set = sum(train_set, [])
 test_set = list()
 for row in fold:
  row_copy = list(row)
 test_set.append(row_copy)
 row_copy[-1] = None
 predicted = algorithm(train_set, test_set, *args)
 actual = [row[-1] for row in fold]
 accuracy = accuracy_metric(actual, predicted)
 scores.append(accuracy)
 return scores

Splitting a dataset involves iterating over each row, checking if the attribute value is below or above the split value and assigning it to the left or right group respectively.

In [ ]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
 left, right = list(), list()
 for row in dataset:
  if row[index] < value:
   left.append(row)
 else:
   right.append(row)
 return left, right

 gini_index() the calculates the Gini index for a list of groups and a list of known class values.

In [ ]:
# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
  # count all samples at split point
 n_instances = float(sum([len(group) for group in groups]))
  # sum weighted Gini index for each group
 gini = 0.0
 for group in groups:
  size = float(len(group))
  # score the group based on the score for each class
 for class_val in classes:
  p = [row[-1] for row in group].count(class_val) / size
 score += p * p
  # weight the group score by its relative size
 gini += (1.0 - score) * (size / n_instances)
 return gini

The get_split() function was used to print out each split point and it’s Gini index as it was evaluated.

In [ ]:
# Select the best split point for a dataset
def get_split(dataset):
 class_values = list(set(row[-1] for row in dataset))
 b_index, b_value, b_score, b_groups = 999, 999, 999, None
 for index in range(len(dataset[0])-1):
  for row in dataset:
   groups = test_split(index, row[index], dataset)
 gini = gini_index(groups, class_values)
 if gini < b_score:
  b_index, b_value, b_score, b_groups = index, row[index], gini, groups
 return {'index':b_index, 'value':b_value, 'groups':b_groups}

to_terminal() that will select a class value for a group of rows. It returns the most common output value in a list of rows.

In [ ]:
# Create a terminal node value
def to_terminal(group):
 outcomes = [row[-1] for row in group]
 return max(set(outcomes), key=outcomes.count)

Once a node is created, we can create child nodes recursively on each group of data from the split by calling the same function again.

Building a decision tree involves calling the above developed get_split() function over and over again on the groups created for each node.

In [ ]:
# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
 left, right = node['groups']
 del(node['groups'])
 # check for a no split
 if not left or not right:
  node['left'] = node['right'] = to_terminal(left + right)
 return
 # check for max depth
 if depth >= max_depth:
  node['left'], node['right'] = to_terminal(left), to_terminal(right)
 return
  # process left child
 if len(left) <= min_size:
  node['left'] = to_terminal(left)
 else:
  node['left'] = get_split(left)
 split(node['left'], max_depth, min_size, depth+1)
# process right child
 if len(right) <= min_size:
  node['right'] = to_terminal(right)
 else:
  node['right'] = get_split(right)
 split(node['right'], max_depth, min_size, depth+1)

build_tree() function
Building the tree involves creating the root node and calling the split() function that then calls itself recursively to build out the whole tree.

In [ ]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
 root = get_split(train)
 split(root, max_depth, min_size, 1)
 return root

 predict() function - Making predictions with a decision tree involves navigating the tree with the specifically provided row of data.

In [ ]:
# Make a prediction with a decision tree
def predict(node, row):
 if row[node['index']] < node['value']:
  if isinstance(node['left'], dict):
   return predict(node['left'], row)
 else:
   return node['left']
 if isinstance(node['right'], dict):
     return predict(node['right'], row)
 else:
     return node['right']

decision_tree() was developed to manage the application of the CART algorithm, first creating the tree from the training dataset, then using the tree to make predictions on a test dataset.

In [ ]:
# Classification Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
 tree = build_tree(train, max_depth, min_size)
 predictions = list()
 for row in test:
  prediction = predict(tree, row)
 predictions.append(prediction)
 return(predictions) 

In [ ]:
# Test CT on Bank Note dataset
seed(1)

In [ ]:
import numpy as np

In [ ]:
# load and prepare data
filename = '/content/BankNote_Authentication.csv'
dataset = load_csv(filename)

In [ ]:
print(' Accuracy: %.3f%%' % np.mean(predict == decision_tree))

 Accuracy: 0.000%
